In [ ]:
import os
import json
import uuid
import httpx
from typing import Tuple


# langgraph_url = os.getenv("LANGGRAPH_URL", "http://localhost:8123")
langgraph_url="http://localhost:8123"


async def create_thread(user_id: str) -> dict:
    """Create a new thread for the given user."""
    try:
        async with httpx.AsyncClient() as client:
            response = await client.post(
                url=f"{langgraph_url}/threads",
                json={
                    "thread_id": str(uuid.uuid4()),
                    "metadata": {
                        "user_id": user_id
                    },
                    "if_exists": "do_nothing"
                }
            )
            response.raise_for_status()

            return response.json()
    except Exception as e:
        print(f"Request failed: {e}")
        raise


async def get_thread_state(thread_id: str) -> dict:
    """Get the state of the thread."""
    try:
        async with httpx.AsyncClient() as client:
            response = await client.get(
                url=f"{langgraph_url}/threads/{thread_id}/state"
            )
            response.raise_for_status()

            return response.json()
    except Exception as e:
        print(f"Request failed: {e}")
        raise


def process_line(line: str, current_event: str) -> str:
    """Process a single data line from the streaming response."""
    try:
        # Process data lines
        if line.startswith("data: "):
            data_content = line[6:]

            if current_event == "messages":
                message_chunk, metadata = json.loads(data_content)
            
                if "type" in message_chunk and message_chunk["type"] == "AIMessageChunk":
                    return message_chunk['content']
                        
                if "tool_calls" in message_chunk:
                    if message_chunk["tool_calls"] and message_chunk["tool_calls"][0]["name"]:
                        tool_name = message_chunk["tool_calls"][0]["name"]
                        tool_args = message_chunk["tool_calls"][0]["args"]

                        tool_call_str = f"\n\n[ TOOL CALL: {tool_name} ]"
                        for arg, value in tool_args.items():
                            tool_call_str += f"\n<{arg}>: \n{value}\n\n"
                        return tool_call_str
                
                # You can handle other event types here
                
            elif current_event == "metadata":
                return

    except Exception as e:
        print(f"Error processing line: {type(e).__name__}: {str(e)}")
        raise


async def get_stream(thread_id: str, message: str):
    """Send a message to the thread and process the streaming response.

    Args:
        thread_id: The thread ID to send the message to
        message: The message content

    Returns:
        str: The complete response from the assistant
    """
    tool_calls = set()
    full_content = ""
    current_event = None

    try:
        async with httpx.AsyncClient() as client:
            async with client.stream(
                "POST",
                url=f"{langgraph_url}/threads/{thread_id}/runs/stream",
                json={
                    "assistant_id": "scout",
                    "input": {
                        "messages": [
                            {"role": "human", "content": message}
                        ]
                    },
                    "stream_mode": "messages-tuple"
                },
                timeout=60.0
            ) as stream_response:
                async for line in stream_response.aiter_lines():
                    if line:
                        # Process event lines
                        if line.startswith("event: "):
                            current_event = line[7:].strip()

                        # Process data lines
                        else:
                            message_chunk = process_line(line, current_event)
                            if message_chunk:

                                start_n_tools = len(tool_calls)
                                if "TOOL CALL" in message_chunk:
                                    tool_calls.add(message_chunk)
                                    if len(tool_calls) > start_n_tools:
                                        print(message_chunk, end="", flush=True)
                                else:
                                    full_content += message_chunk
                                    print(message_chunk, end="", flush=True)

        return full_content
    except Exception as e:
        print(f"Error in get_stream: {type(e).__name__}: {str(e)}")
        raise


async def main():
    try:
        # Create a thread
        response = await create_thread(user_id="kenny")
        thread_id = response["thread_id"]

        current_chart = None
        # Stream responses
        while True:
            user_input = input("User: ")
            if user_input.lower() in ["exit", "quit"]:
                break

            print(f"\n---- User ---- \n\n{user_input}\n")

            print(f"---- Assistant ---- \n")
            # Get the response using our simplified get_stream function
            result = await get_stream(thread_id, user_input)

            # check state after run
            thread_state = await get_thread_state(thread_id)

            if "chart_json" in thread_state["values"]:
                chart_json = thread_state["values"]["chart_json"]
                if chart_json and chart_json != current_chart:
                    # render any charts generated
                    import plotly.io as pio
                    fig = pio.from_json(chart_json)
                    fig.show()

                    current_chart = chart_json
            print("")

    except Exception as e:
        print(f"Error: {type(e).__name__}: {str(e)}")
        raise


if __name__ == "__main__":
    import asyncio
    import nest_asyncio
    nest_asyncio.apply()

    print("\n###\n\nGreetings!\n\nTry asking Scout a question about the company data.\n\n###\n\n")

    asyncio.run(main())


###

Greetings!

Try asking Scout a question about the company data.

###



---- User ---- 

what was the total revenue in 2024?

---- Assistant ---- 

Plan:
To find the total revenue in 2024, I will query the transactions table to sum the amount_usd for all transactions where the transaction_date falls within the year 2024.

I will write a SQL query to:
- Filter transactions with transaction_date between '2024-01-01' and '2024-12-31'
- Sum the amount_usd for these transactions

I will execute this query and provide the total revenue for 2024.The total revenue in 2024 was $11,252.90. If you need any further breakdown or analysis, please let me know!

---- User ---- 

can you plot the total revenue by month in 2024?

---- Assistant ---- 

Plan:
I will create a SQL query to aggregate the total revenue by month for the year 2024. This will involve:
- Filtering transactions for the year 2024
- Grouping by the month of the transaction_date
- Summing the amount_usd for each month

Then, I 



---- User ---- 

can you make it a line plot?

---- Assistant ---- 

Plan:
I will modify the existing visualization code to use a line plot instead of a bar chart. The SQL query remains the same since it already aggregates total revenue by month for 2024.

I will regenerate the visualization with the updated plot type as a line plot.

[ TOOL CALL: query_db ]
<query>: 
SELECT SUM(amount_usd) AS total_revenue_2024
FROM transactions
WHERE transaction_date >= '2024-01-01' AND transaction_date < '2025-01-01';



[ TOOL CALL: generate_visualization ]
<name>: 
total_revenue_by_month_2024


<sql_query>: 
SELECT DATE_TRUNC('month', transaction_date) AS month, SUM(amount_usd) AS total_revenue
FROM transactions
WHERE transaction_date >= '2024-01-01' AND transaction_date < '2025-01-01'
GROUP BY month
ORDER BY month;


<plotly_code>: 
fig = px.bar(df, x='month', y='total_revenue', title='Total Revenue by Month in 2024')
fig.update_layout(xaxis_title='Month', yaxis_title='Total Revenue ($)', xaxis



---- User ---- 

can you make it cumulative?

---- Assistant ---- 

Plan:
To create a cumulative line plot of total revenue by month in 2024, I will:
- Use the same SQL query to get monthly total revenue.
- In the visualization code, calculate the cumulative sum of the total revenue.
- Plot the cumulative revenue over the months as a line plot.

I will regenerate the visualization accordingly.

[ TOOL CALL: query_db ]
<query>: 
SELECT SUM(amount_usd) AS total_revenue_2024
FROM transactions
WHERE transaction_date >= '2024-01-01' AND transaction_date < '2025-01-01';



[ TOOL CALL: generate_visualization ]
<name>: 
total_revenue_by_month_2024


<sql_query>: 
SELECT DATE_TRUNC('month', transaction_date) AS month, SUM(amount_usd) AS total_revenue
FROM transactions
WHERE transaction_date >= '2024-01-01' AND transaction_date < '2025-01-01'
GROUP BY month
ORDER BY month;


<plotly_code>: 
fig = px.bar(df, x='month', y='total_revenue', title='Total Revenue by Month in 2024')
fig.update_layou



---- User ---- 

can you overlay the revenue from 2023?

---- Assistant ---- 

Plan:
To overlay the revenue from 2023 on the cumulative revenue plot for 2024, I will:
- Write a SQL query to get monthly total revenue for both 2023 and 2024.
- Calculate cumulative revenue for each year separately in the visualization code.
- Create a line plot with two lines: one for cumulative revenue in 2023 and one for cumulative revenue in 2024.

I will proceed with this approach.

[ TOOL CALL: query_db ]
<query>: 
SELECT SUM(amount_usd) AS total_revenue_2024
FROM transactions
WHERE transaction_date >= '2024-01-01' AND transaction_date < '2025-01-01';



[ TOOL CALL: generate_visualization ]
<name>: 
total_revenue_by_month_2024


<sql_query>: 
SELECT DATE_TRUNC('month', transaction_date) AS month, SUM(amount_usd) AS total_revenue
FROM transactions
WHERE transaction_date >= '2024-01-01' AND transaction_date < '2025-01-01'
GROUP BY month
ORDER BY month;


<plotly_code>: 
fig = px.bar(df, x='month', y



---- User ---- 

the lines are offset on the x axis which makes sense given the timeline. But I want to look at the lines overlayed on the same timeline by month only so Jan through Dec so I can compare them by month

---- Assistant ---- 

Plan:
To compare the cumulative revenue for 2023 and 2024 by month on the same timeline (Jan through Dec), I will:
- Modify the SQL query to extract the month number and year.
- Calculate cumulative revenue by year and month number.
- Plot the cumulative revenue with the x-axis as month number (1 to 12) and overlay the lines for 2023 and 2024.

This will allow direct month-to-month comparison across the two years. I will proceed with this approach.

[ TOOL CALL: query_db ]
<query>: 
SELECT SUM(amount_usd) AS total_revenue_2024
FROM transactions
WHERE transaction_date >= '2024-01-01' AND transaction_date < '2025-01-01';



[ TOOL CALL: generate_visualization ]
<name>: 
total_revenue_by_month_2024


<sql_query>: 
SELECT DATE_TRUNC('month', transactio



---- User ---- 

add a second y-axis that shows the cumulative number of customers for both years by month

---- Assistant ---- 

Plan:
To add a second y-axis showing the cumulative number of customers by month for both years, I will:
- Modify the SQL query to also count distinct customers per month and year.
- Calculate cumulative sums for both revenue and customer counts by year and month.
- Create a plot with two y-axes: one for cumulative revenue and one for cumulative customers.
- Overlay the lines for 2023 and 2024 for both metrics.

I will proceed with this approach.

[ TOOL CALL: query_db ]
<query>: 
SELECT SUM(amount_usd) AS total_revenue_2024
FROM transactions
WHERE transaction_date >= '2024-01-01' AND transaction_date < '2025-01-01';



[ TOOL CALL: generate_visualization ]
<name>: 
total_revenue_by_month_2024


<sql_query>: 
SELECT DATE_TRUNC('month', transaction_date) AS month, SUM(amount_usd) AS total_revenue
FROM transactions
WHERE transaction_date >= '2024-01-01' AND 



---- User ---- 

what else do you think I should look at to compare 2023 vs 2024 performance?

---- Assistant ---- 

To comprehensively compare 2023 vs 2024 performance, here are some additional analyses and metrics you might consider exploring:

1. Monthly New Customer Acquisition:
   - Track the number of new customers acquired each month in both years.
   - This helps understand growth in customer base.

2. Average Revenue per Customer (ARPC):
   - Calculate monthly average revenue per active customer.
   - This reveals changes in customer spending behavior.

3. Customer Retention and Churn Rates:
   - Analyze how many customers from previous months/years continue transacting.
   - Identify if retention improved or worsened.

4. Transaction Volume:
   - Compare the number of transactions per month or year.
   - Helps understand engagement frequency.

5. Revenue by Transaction Type:
   - Breakdown revenue by different transaction types to see if product mix changed.

6. Top Creator



---- User ---- 

why is there such a disparity between May 2023 and Feb 2023?

---- Assistant ---- 

To understand the disparity in ARPC between May 2023 and February 2023, we should look deeper into the underlying factors that could cause this difference:

1. Total Revenue: Check if total revenue in May 2023 was significantly lower than in February 2023.
2. Number of Active Customers: Check if the number of active customers in May 2023 was significantly higher than in February 2023.
3. Transaction Volume and Size: Look at the number of transactions and average transaction size in these months.
4. Customer Segments: See if there were changes in customer segments or high-value customers in these months.
5. Promotions or Discounts: Check if any promotions or discounts were running in May 2023 that could have affected revenue or customer behavior.

I will start by comparing total revenue, active customers, and transaction count for February 2023 vs May 2023.

[ TOOL CALL: query_db ]
<qu